In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.731 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", 
    map_eos_token = True
)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("json", data_files="conversation-with-system-prompt.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Will map <|im_end|> to EOS = </s>.


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1712 [00:00<?, ? examples/s]

In [4]:
dataset['text'][5]

'<|im_start|>system\nYour task is to mix ambient sounds by composing a set of ambient audio tracks.\nYou can assign each track a <volume> between 0 and 1.\nYou can only use each track once.\nTracks will loop indefinitely unless <random> is set to true.\nIn this case, the track will repeat randomly every <random_unit> for <random_counter> times.\nYour response should be a JSON object of the following format:\n---\n[\n  {\n    "name": "<track name>",\n    "volume": "<the volume>",\n    "random": "<true or false>",\n    "random_counter": "<number of times to play the sound every random_unit if random = true>",\n    "random_unit": "<1m|10m|1h>",\n  },\n  ...\n]\n---\nI will provide descriptions of the enviroment and mood I\'d like to create an ambient mix for in my next prompt.\n<|im_end|>\n<|im_start|>user\nSpring Thunderstorm at Night: A slow thunderstorm on a spring night, with rain, crickets, and startling hits nearby.<|im_end|>\n<|im_start|>assistant\n[{"name": "Rain on a tin roof", "

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "unsloth_mistral-7b-instruct-v0.2-bnb-4bit_with-system-prompt",
    ),
)

Map (num_proc=2):   0%|          | 0/1712 [00:00<?, ? examples/s]

In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,712 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss


## Memory Stats

In [7]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

GPU = NVIDIA RTX A4000. Max memory = 15.731 GB.
5.68 GB of memory reserved.
359.9592 seconds used for training.
6.0 minutes used for training.
Peak reserved memory = 5.68 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 36.107 %.
Peak reserved memory for training % of max memory = 0.0 %.


## Inference

In [8]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def evaluate(prompt: str, include_system_prompt=True):
    if include_system_prompt:
        assert dataset['messages'][0][0]['role'] == 'system'
        system_prompt = dataset['messages'][0][0]['content']

        messages = [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': prompt}
        ]
    else:
        messages = [
            {'role': 'user', 'content': prompt}
        ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = False,
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 2048, use_cache = True)
    return tokenizer.batch_decode(outputs)

In [9]:
evaluate('A train ride through a rainy night')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<|im_start|>system\nYour task is to mix ambient sounds by composing a set of ambient audio tracks.\nYou can assign each track a <volume> between 0 and 1.\nYou can only use each track once.\nTracks will loop indefinitely unless <random> is set to true.\nIn this case, the track will repeat randomly every <random_unit> for <random_counter> times.\nYour response should be a JSON object of the following format:\n---\n[\n  {\n    "name": "<track name>",\n    "volume": "<the volume>",\n    "random": "<true or false>",\n    "random_counter": "<number of times to play the sound every random_unit if random = true>",\n    "random_unit": "<1m|10m|1h>",\n  },\n  ...\n]\n---\nI will provide descriptions of the enviroment and mood I\'d like to create an ambient mix for in my next prompt.\n<|im_end|> \n<|im_start|>user\nA train ride through a rainy night<|im_end|> \n<|im_start|>assistant\n[{"name": "Rain on a tin roof", "volume": 0.3, "random": false, "random_counter": 0, "random_unit": "1h"}, {"nam

## Saving the Model

In [ ]:
model.save_pretrained_gguf("unsloth_mistral-7b-instruct-v0.2-bnb-4bit_no-system-prompt_gguf-q4_k_m", tokenizer, quantization_method = "q4_k_m")